In [ ]:
import requests
import re
import urllib 
from bs4 import BeautifulSoup
import string
import pandas as pd
import numpy as np
import datetime
import _datetime

In [ ]:
current_datetime1 = datetime.datetime.now()
temporary_links = []
primary_links = []
product_links = []
titles = []
prices = []
marks=[]
articles=[]
categories=[]
product_links1=[]
url = 'https://www.mactak.ru/catalog/'
r = requests.get(url)
html_content = r.text
soup = BeautifulSoup(html_content,'html.parser')
for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_="open_ul_akkordeon"):
        temporary_links.append(link.get('href'))

In [ ]:
k=0
print(len(temporary_links))
for j in temporary_links:
    url='https://www.mactak.ru'+j
    try:
        r = requests.get(url)
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
        print("pobeda")
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')
    if soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_="last-page")==[]:
        last_pages=2
    else:
        last_pages=soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_="last-page")[0].text
    for i in range(1, int(last_pages) + 1):
        url='https://www.mactak.ru'+j+'?PAGEN_1='+str(i)
        try:
            r = requests.get(url)
        except requests.exceptions.ConnectionError:
            r.status_code = "Connection refused"
            print("pobeda")
        html_content = r.text
        soup = BeautifulSoup(html_content,'html.parser')
        for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_="picture"):
                product_links.append(link.get('href'))
        print(i,'i')
    print(j,'j')
print(len(product_links))
for product_link in product_links:
    url = 'https://www.mactak.ru' + product_link
    try:
        r = requests.get(url)
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
        print("pobeda")
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')

    title = soup.findAll('h1')[0].text
    titles.append(title)

    if soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_="mark")!=[]:
        mark = soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_="mark")[0].text
    else:
        mark =[]
    marks.append(mark)

    article = soup.findAll('span',class_="article")[0].text.split()[-1]
    articles.append(article)

    price=soup.findAll('span',class_="cur-price norm_price")[0].text
    prices.append(price)

    category=soup.findAll('span', attrs={'itemprop': 'name','class':[]})[-1].text
    categories.append(category)
    k=k+1
    print(k)
    product_links1.append(url)
#product_links1 =[ 'https://www.mactak.ru%s'%x for x in product_links ]

table = pd.DataFrame(list(zip(titles, marks, articles,prices, categories,product_links1)), columns =['Наименование товара', 'Бренд', 'Артикул','Цена, руб.','Категория','Ссылка на товар']) 

table.to_excel('mactak.xlsx')

current_datetime2 = datetime.datetime.now()
print (current_datetime2 - current_datetime1)
    